# 단일 PDF 파일 챗봇 샘플 코드

### FAISS 활용


### 환경설정

In [ ]:
# 패키지 설치 후 커널 리스타트 해주세요.

!pip install boto3 langchain langchain_aws langchain_community -U

In [4]:
import os
import boto3
import time
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_aws import ChatBedrock

# Bedrock 클라이언트 설정
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"  # 사용하는 리전으로 변경하세요
)


In [9]:
%%time

# PDF 로드 및 텍스트 분할
loader = PyPDFLoader("./rag_data/Steel Business Briefing_15 Jul 2024.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Bedrock 임베딩 설정
embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v1" # Titan Embeddings V1
)

# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents(texts, embeddings)

# Claude 3 Sonnet LLM 설정
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 4096, 
        "temperature": 0.5
    },
    
)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    verbose = True,
    retriever=vectorstore.as_retriever()
)

# 쿼리 실행 함수
def query_pdf(question):
    response = qa_chain.run(question)
    return response


CPU times: user 1.61 s, sys: 0 ns, total: 1.61 s
Wall time: 17.9 s


In [10]:
%%time

# 사용자 쿼리
question = """
PDF 문서의 주요 내용은 무엇인가요?
"""

answer = query_pdf(question)
print(f"질문: {question}")
print(f"답변: {answer}")
print()



> Entering new RetrievalQA chain...

> Finished chain.
질문: 
PDF 문서의 주요 내용은 무엇인가요?

답변: 이 PDF 문서는 S&P 글로벌 커모디티 인사이트(S&P Global Commodity Insights)의 2024년 7월 15일자 일일 브리핑 보고서인 것 같습니다. 주요 내용은 다음과 같습니다:

- 아시아 열연코일 가격이 중국 공산당 3중 전회에 대한 기대감으로 상승세를 보임

- 클리브랜드 클리프스의 U.S. 스틸 인수 제안이 거부되고 대신 일본 신일본제철이 140억 달러에 U.S. 스틸을 인수하기로 함

- 남아프리카 대통령이 EU의 탄소국경조정메커니즘(CBAM)으로 인해 탄소 집약적 산업에 경제적 위험이 있음을 지적

- 유럽 열연코일 가격 평가 및 근거 설명

- 유럽 탄소가격을 반영한 열연코일 가격 평가 및 근거 설명

요약하면 철강 제품 가격 동향, 주요 기업 인수합병 소식, 탄소세 정책 영향 등 철강/원자재 산업 전반의 뉴스와 가격 정보를 담고 있습니다.

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 8.53 s
